In [161]:
from bs4 import BeautifulSoup
from unidecode import unidecode

import pandas as pd
import numpy as np

import string
import requests

from time import sleep

import warnings; warnings.filterwarnings(action='ignore')

# RECLAME AQUI

In [10]:
HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'}

def get_complains(company_id, company_name, n_complains):

    complains = {'company_name': [], 'description': [], 'href': [], 'full_complain': []}
    
    for n in range(0, n_complains, 10):
        url = f'https://iosearch.reclameaqui.com.br/raichu-io-site-search-v1/query/companyComplains/10/{n}'
        res = requests.get(url, params={'company': company_id}, headers=HEADERS).json()

        for x in range(10):
            data = res['complainResult']['complains']['data']
            
            description = data[x]['description']
            soup = BeautifulSoup(description, 'lxml').text
            complains['description'].append(soup)
            
            id_ = data[x]['id']
            title = data[x]['title']
            clean_title = title.translate(str.maketrans('', '', string.punctuation))
            href = clean_title.lower().strip() + '_' + id_
            href = unidecode(href.replace(' ', '-'))
            complains['href'].append(href)
            
            complains['company_name'].append(company_name)
            
            complains['full_complain'].append('')
            
    df = pd.DataFrame(complains)
    
    for i, company_name, href in zip(df.index, df.company_name, df.href):
        try:
            url = f"https://www.reclameaqui.com.br/{company_name}/{href}/"
            res = requests.get(url, headers=HEADERS)

            soup = BeautifulSoup(res.text, 'lxml')
            complain = soup.find('p', {'class': 'lzlu7c-17 fXwQIB'}).text
            df.iloc[i]['full_complain'] = complain
        except:
            df.drop(index=i, inplace=True)

            
    return df

# INFOS

In [37]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By

In [61]:
firefox_options = Options()
firefox_options.add_argument('--headless')

p = '/home/vitor/drivers/'

In [162]:
# INFORMACOES DO TRAFEGO
def get_info(company):
    firefox_options = Options()
    firefox_options.add_argument('--headless')
    p = '/home/vitor/drivers/'
    driver = webdriver.Firefox(p, options=firefox_options)
    
    driver.get(f'https://www.similarweb.com/website/{company}/')
    
    sleep(0.5)
    
    result = {
        'company_name': [company],
        'brazilian_visitors': [-1]
    }

    info = driver.find_elements(By.CLASS_NAME, 'engagement-list__item')
    
    bounce_rate = float(info[1].text.split('\n')[1][:-1]) / 100
    result['bounce_rate'] = [round(bounce_rate, 3)]
    
    visit_dur = int(info[3].text.split('\n')[1][3:5])
    result['visit_duration'] = [visit_dur]
    
    if info[0].text.split('\n')[1][-1] == 'M':
        total_visits = float(info[0].text.split('\n')[1][:-1]) * 1_000_000
    else:
        total_visits = float(info[0].text.split('\n')[1][:-1]) * 1_000
        
    country = driver.find_elements(By.CLASS_NAME, 'wa-geography__country-name')
    pct = driver.find_elements(By.CLASS_NAME, 'wa-geography__country-traffic-value')
    for c, p in zip(country, pct):
        c_text = c.text
        p_text = p.text
        
        if c_text == 'Brazil':
            percentage = float(p_text[:-1]) / 100
            br_visits = round(total_visits * percentage)
            result['brazilian_visitors'] = [br_visits]
        else:
            continue
    
    sex = driver.find_elements(By.CLASS_NAME, 'wa-demographics__gender-legend')[0].text
    sex_text = sex.split('\n')
    male = float(sex_text[3][:-1]) / 100
    female = float(sex_text[1][:-1]) / 100
    result['male'] = [round(male, 3)]
    result['female'] = [round(female, 3)]
    
    svg_eles = driver.find_elements(By.CLASS_NAME, 'highcharts-root')
    svg = svg_eles[3]
    svg_text = svg.text
    
    idades = svg_text.split('\n')[:6]
    labels = ['18-24', '25-34', '35-44', '45-54', '55-64', '65+']
    for i, l in zip(idades, labels):
        i_pct = round((float(i[:-1]) / 100), 3)
        result[l] = [i_pct]
    
    driver.quit()
    
    df = pd.DataFrame(result)
    
    sleep(0.5)
    
    return df

In [157]:
%%time

res = get_info('udemy.com')
res

CPU times: user 419 ms, sys: 27.8 ms, total: 447 ms
Wall time: 29.4 s


,company_name,brazilian_visitors,bounce_rate,visit_duration,male,female,18-24,25-34,35-44,45-54,55-64,65+
0,udemy.com,6604220,0.369,11,0.664,0.335,0.271,0.367,0.183,0.096,0.053,0.03


In [163]:
test_comps = ['udemy.com', 'coursera.org']
df = pd.concat((get_info(comp) for comp in test_comps), ignore_index=True)

df

,company_name,brazilian_visitors,bounce_rate,visit_duration,male,female,18-24,25-34,35-44,45-54,55-64,65+
0,udemy.com,6604220,0.369,11,0.664,0.335,0.271,0.367,0.183,0.096,0.053,0.030
1,coursera.org,1756040,0.360,11,0.557,0.443,0.286,0.353,0.177,0.098,0.055,0.032
